# Stochastic volatility model

Notebook for the estimation of the model parameters

### Mount Drive

In [ ]:
# mount my drive
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wFN-hOA1L4E423jscCj5Tie48U-OmeRZ-ccC7AMDM7zSkR3arcQ7x8
Mounted at /content/drive


The following path has to be adjusted if necessary:

In [ ]:
%run "/content/drive/My Drive/Colab Notebooks/Utils.ipynb"
%run "/content/drive/My Drive/Colab Notebooks/Plot_fcts.ipynb"

### Imports

In [ ]:
! pip install qmcpy --quiet

     |████████████████████████████████| 368kB 7.6MB/s 


In [ ]:
import numpy as np
import qmcpy

### Optimisation

In [ ]:
np.random.seed(11)

Set parameters:

In [ ]:
m = 2**10                                         # number of true data points
n = 2**9                                          # number of simulated data points
theta_star = np.array([0.98,0.65,0.15])           # true theta
d = 30                                            # dimensions of data
p = theta_star.shape[0]                           # dimensions of parameter space
l = -1                                            # lengthscale (l=-1 for median heuristic)
method_gd = 'SGD'                                 # SGD or NSGD
eta = 0.01                                         # step size
max_it = 5000                                     # number of descent steps
start = np.zeros(p)                               # start values

Define QMC sequence:

In [ ]:
qmc = qmcpy.Halton(2+2*d)

Sample from the SV model:

In [ ]:
y,_,_ = sample_sv('MC',m,d,theta_star)

Gradient descent:

In [ ]:
theta_mc,loss_mc = optim('sv','MC',method_gd,eta,max_it,l,n,m,d,p,y,start,sparse=True)

iteration: 1000 
loss:      0.000635 
estimate:  [-0.002  0.665  0.166] 
gradient:  [ 9.493e-05 -7.866e-03 -2.302e-03]
iteration: 2000 
loss:      -6.73e-05 
estimate:  [-0.002  0.668  0.171] 
gradient:  [ 9.523e-05 -3.461e-03 -1.047e-03]
iteration: 3000 
loss:      0.0005551 
estimate:  [-0.002  0.663  0.176] 
gradient:  [0.    0.005 0.001]
iteration: 4000 
loss:      0.0004452 
estimate:  [-0.002  0.663  0.181] 
gradient:  [ 0.    -0.017 -0.005]
iteration: 5000 
loss:      0.0004117 
estimate:  [-0.002  0.662  0.185] 
gradient:  [-5.667e-05 -1.970e-02 -5.016e-03]
-------------------------------------------
final loss:        0.0004117 
final estimate:    [-0.002  0.662  0.185] 
total iterations:  5000
Elapsed time is 38209.271783 seconds.


In [ ]:
theta_qmc,loss_qmc = optim('sv','QMC',method_gd,eta,max_it,l,n,m,d,p,y,start,sparse=True)

iteration: 1000 
loss:      -0.0003747 
estimate:  [-0.004  0.672  0.168] 
gradient:  [ 6.632e-05 -1.944e-03 -4.389e-04]
iteration: 2000 
loss:      -0.0003775 
estimate:  [-0.005  0.674  0.168] 
gradient:  [ 4.897e-05 -1.549e-05  5.659e-05]
iteration: 3000 
loss:      -0.0003776 
estimate:  [-0.005  0.675  0.167] 
gradient:  [ 4.899e-05 -1.394e-05  5.696e-05]
iteration: 4000 
loss:      -0.0003777 
estimate:  [-0.006  0.675  0.167] 
gradient:  [ 4.902e-05 -1.393e-05  5.693e-05]
iteration: 5000 
loss:      -0.0003777 
estimate:  [-0.006  0.675  0.166] 
gradient:  [ 4.905e-05 -1.391e-05  5.691e-05]
-------------------------------------------
final loss:        -0.0003777 
final estimate:    [-0.006  0.675  0.166] 
total iterations:  5000
Elapsed time is 41980.836092 seconds.


In [ ]:
theta_rqmc,loss_rqmc = optim('sv','RQMC',method_gd,eta,max_it,l,n,m,d,p,y,start,sparse=True)

### Visualise results

In [ ]:
it = 10000

Plot the loss:

In [ ]:
plot_loss(it,d,(12,6),loss_mc,loss_qmc,loss_rqmc,'MC','QMC','RQMC')

Plot the estimates:

In [ ]:
plot_estimates(2,p,d,(12,((p//2)+(p%2))*6),max_it,theta_mc,theta_qmc,theta_rqmc,theta_star,'MC','QMC','RQMC')

Plot the MSE:

In [ ]:
mse_mc,mse_qmc,mse_rqmc = mse(max_it,p,theta_mc,theta_qmc,theta_rqmc,theta_star)

In [ ]:
plot_mse(2,p,d,(12,((p//2)+(p%2))*6),max_it,mse_mc,mse_qmc,mse_rqmc,'MC','QMC','RQMC','sv')